# Curso Aprendizaje Automático 

## Tarea Programada 03

### Estudiantes:

- **Carlos Jesus Abarca Murillo**
- **Maria Paula Castillo Espinoza**

---

In [17]:
import pandas as pd
import unicodedata
from openpyxl import load_workbook

Inicialmente se cargan y preprocesan los datos del INEC y se preprocesan para que provincia, cantón y ddistrito estén en su propia columna y sea más fácil hacer join con los datos del OIJ.

In [18]:
# --- Parámetros ajustables que dependen del archivo ---
XLSX_PATH   = "DatosINEC.xlsx"   # Ruta al archivo INEC convertido a .xlsx
SHEET_NAME  = "Cuadro 22"        # Nombre de la hoja a procesar
HDR_ROW_1   = 6                  # Primera fila del encabezado (1-indexed en Excel)
HDR_ROW_2   = 7                  # Segunda fila del encabezado (1-indexed)
DATA_START  = 9                  # Fila donde arrancan los datos (1-indexed)

# --- Utilidades ---
def norm_text(s: str) -> str:
    #Quita acentos, espacios y pasa a minúsculas.
    if s is None:
        return ""
    s = str(s).strip()
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    s = " ".join(s.split())
    return s.lower()

PROVINCIAS_CR = {
    "san jose", "alajuela", "cartago", "heredia", "guanacaste", "puntarenas", "limon"
}

# --- Carga del libro ---
wb = load_workbook(XLSX_PATH, data_only=True)
if SHEET_NAME not in wb.sheetnames:
    raise ValueError(f"La hoja '{SHEET_NAME}' no existe. Hojas disponibles: {wb.sheetnames}")
ws = wb[SHEET_NAME]

max_row = ws.max_row
max_col = ws.max_column

# --- 1) Construir nombres de columnas desde filas 6 y 7 ---
# Regla:
# - Si hay texto en la fila 7, úsalo.
# - Si no, usa el de la fila 6.
# - Si hay en ambas y no son iguales, concatena.
col_names = {}
for c in range(1, max_col + 1):
    v1 = ws.cell(row=HDR_ROW_1, column=c).value
    v2 = ws.cell(row=HDR_ROW_2, column=c).value

    t1 = (str(v1).strip() if v1 is not None else "")
    t2 = (str(v2).strip() if v2 is not None else "")

    if t2:
        name = t2
    elif t1:
        name = t1
    else:
        name = f"Unnamed_{c}"

    # Si hay ambas y son diferentes y ninguna es vacía, combinar evitando duplicados
    if t1 and t2 and t1 != t2:
        n1 = norm_text(t1)
        n2 = norm_text(t2)
        if n2 not in n1 and n1 not in n2:
            name = f"{t1} - {t2}"

    col_names[c] = name

# Identificar la columna de ubicación (Provincia, cantón y distrito)
loc_col_idx = None
for c, name in col_names.items():
    v = norm_text(name)
    if ("provincia" in v) and ("canton" in v) and ("distrito" in v):
        loc_col_idx = c
        break
if loc_col_idx is None:
    raise RuntimeError("No se encontró la columna de 'Provincia, cantón y distrito' en los encabezados combinados.")

# --- 2) Reconstruir jerarquía por formato (negrita) ---
rows_out = []
curr_prov = None
curr_canton = None

for r in range(DATA_START, max_row + 1):
    cell = ws.cell(row=r, column=loc_col_idx)
    text_raw = cell.value

    # Saltar filas vacías o separadores
    if text_raw is None or str(text_raw).strip() == "":
        continue

    is_bold = bool(cell.font and cell.font.bold)
    txt_norm = norm_text(text_raw)
    text_clean = str(text_raw).strip()

    if is_bold:
        # Si es negrita, puede ser Provincia o Cantón
        # Regla: si es provincia oficial y distinta de la actual -> Provincia.
        # Si es igual a la provincia actual o no es provincia -> Cantón.
        if (txt_norm in PROVINCIAS_CR) and (curr_prov != text_clean):
            curr_prov = text_clean
            curr_canton = None
        else:
            curr_canton = text_clean
        continue  # filas en negrita NO son distritos

    # Si no es negrita => es Distrito, hereda provincia y cantón vigentes
    distrito = text_clean
    prov = curr_prov
    canton = curr_canton

    # Construir el registro con Provincia, Cantón, Distrito + métricas de la fila
    row = {
        "Provincia": prov,
        "Cantón": canton,
        "Distrito": distrito
    }
    for c in range(1, max_col + 1):
        if c == loc_col_idx:
            continue
        row[col_names[c]] = ws.cell(row=r, column=c).value
    rows_out.append(row)

# --- 3) A DataFrame y limpieza ligera ---
df_out = pd.DataFrame(rows_out)

for col in ["Provincia", "Cantón", "Distrito"]:
    if col in df_out.columns:
        df_out[col] = (
            df_out[col]
            .astype("string")
            .str.replace(r"\s+", " ", regex=True)
            .str.strip()
        )

# Quitar columnas completamente vacías y las últimas 3 columnas del archivo ya que no son de valor para este ejercicio
non_empty_ratio = df_out.notna().mean()
keep_cols = [c for c in df_out.columns if (non_empty_ratio[c] > 0.0) or (c in ["Provincia", "Cantón", "Distrito"])]
df_out = df_out.iloc[:, :-3]

# --- 4) Guardar ---
OUT_XLSX = "DatosINEC_procesado.xlsx"
df_out.to_excel(OUT_XLSX, index=False, sheet_name="INEC_Distritos")

print(f"Listo. Guardado en: {OUT_XLSX}")
print(df_out.head(10))


Listo. Guardado en: DatosINEC_procesado.xlsx
  Provincia    Cantón                   Distrito Unnamed_1  \
0  San José  San José                     Carmen      None   
1  San José  San José                     Merced      None   
2  San José  San José                   Hospital      None   
3  San José  San José                   Catedral      None   
4  San José  San José                     Zapote      None   
5  San José  San José  San Francisco de Dos Ríos      None   
6  San José  San José                      Uruca      None   
7  San José  San José               Mata Redonda      None   
8  San José  San José                      Pavas      None   
9  San José  San José                    Hatillo      None   

   Población de 15 años y más  Tasa neta de participación  Tasa de ocupación  \
0                        2431                   56.314274          54.792267   
1                        9655                   59.243915          56.996375   
2                       15096   

Ahora que se tienen ambos archivos preprocesados se procede a unirlos por distrito, en este caso de decide unirlos por Distrito, Canton y Provincia ya que en ocasiones el nombre del distrito se puede repetir para varios cantones.

In [20]:
# === Configura tus archivos ===
PATH_INEC = "DatosINEC_procesado.xlsx"     # archivo INEC ya procesado
PATH_OIJ  = "Delitos2011.xlsx"             # archivo OIJ 2011
SHEET_INEC = "INEC_Distritos"              
SHEET_OIJ  = "Sheet2"                      
OUT_FILE   = "Dataset_Full.xlsx"

# --- Función para normalizar texto (quita tildes, minúsculas, trim) ---
def norm_text(s):
    if pd.isna(s):
        return s
    s = str(s).strip()
    s = " ".join(s.split())
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("utf-8")
    return s.lower()

# --- Cargar ---
df1 = pd.read_excel(PATH_INEC, sheet_name=SHEET_INEC)
df2 = pd.read_excel(PATH_OIJ,  sheet_name=SHEET_OIJ)

# Eliminar columnas sin encabezado
df1 = df1.loc[:, ~df1.columns.str.contains(r'^Unnamed', case=False)]
df2 = df2.loc[:, ~df2.columns.str.contains(r'^Unnamed', case=False)]

# --- Limpieza en llaves de join ---
for col in ["Provincia", "Cantón", "Distrito"]:
    if col in df1.columns: df1[col] = df1[col].apply(norm_text)
    if col in df2.columns: df2[col] = df2[col].apply(norm_text)

# --- Merge por 3 columnas ---
df_join = pd.merge(
    df1, df2,
    on=["Provincia", "Cantón", "Distrito"],
    how="inner",     # se escoje inner para conservar solo aquellas filas que hagan match en ambos archivos         
    suffixes=("_INEC", "_OIJ")
)

# --- Guardar ---
df_join.to_excel(OUT_FILE, index=False)

# --- Resumen rápido ---
print("Filas INEC:", len(df1))
print("Filas OIJ :", len(df2))
print("Filas join:", len(df_join))
print("Guardado en:", OUT_FILE)


Filas INEC: 472
Filas OIJ : 459
Filas join: 410
Guardado en: Dataset_Full.xlsx
